## Problem Set 2

_csc427, semester 212
<br>
university of miami
<br>
date: 10 february 2021
<br>
update: 10 february 2021_

<p>
(c) 2021 burton rosenberg
    <br>license below

---

### Student Name: Temuulen Ganbold csc427 ps2 

---

### Introduction to nondeterminism

The question at hand is what can be computed given a particular machine model. This question has been reduced to set recognition problem, what subsets of some string set $\Sigma^*$ can be defined by and recognized by Finite Automata. One way of defining such sets is with Regular Expressions.

A Regular Expression (RE) a way of defining a subset of $\Sigma$ by declaring that any single letter of $\Sigma$ is an RE, the empty string and empty set are RE's, and then from this finitary basis, also sets defined by union, concatentation and (Kleene) star are RE's.

The point is, this <em>generates</em> sets which are exactly the sets that can be <em>recognized</em> by (deterministic) finite automata. That is, given an RE $\Omega$ we can define a FA $M$ such that for any $\sigma \in \Sigma^*$ generated $\Omega$ it is accepted by $M$, and if that $\sigma$ is not generated by $\Omega$ it is not accepted by $M$.

However, the situation is difficult. We have seen situations involving concatentation where the machine is not straightforward, as the machine must search among possibilities in its computation. A <em>Nondeterministic Finite Automata</em> captures this notion of computation, by proposing a machine that is free to branch, guess, or search possibilities, when exploring computation pathways.



### The NFA

We go right ahead to show code for an NFA (nondeterministic finite automata). Our implementation takes a position on nature of nondeterminism, which later on I declare to be the Schroedinger approach. We will trace alternative computations in a single computation trajectory.

The most important differences in the code for the nondeterministic FA compared to the deterministic FA are,

1. Current state is a set of states (new tokens can appear).
1. It is not an error if a key is missing. A missing key is is equivalent to a mapping for that key to the empty state set (a token can disappear).
3. The presence of epsilon moves. These are state transitions can on a special epsilon character, which can be inserted anywhere with any multiplicity in the input string.

The rules of computation are as follows.

1. An epsilon insertion to a string $\sigma \in \Sigma$ is any string $\hat{\sigma}\in \Sigma \cup \{\,\epsilon\,\}$ such that removing all $\epsilon$ from $\hat{\sigma}$ gives $\sigma$.
2. A run on a string is a list of states beginning with the start state and consistent with an epsilon insertion of the string and the transition function.
3. The NFA accepts if there is a run on the string that ends on an accepting state.

Here is a more detailed description of what computations are consistent with the transition function. 

Let the epsilon insertion for input be the string $w$ be written letter by letter as,

$$
w = w_1 w_2 \ldots w_n
$$

where $w_i$ could be a letter in $\Sigma$ or could be $\epsilon$, and let the computation $C$ be written as a sequence of states, 

$$
C = S_1 S_2 \ldots S_{n+1}.
$$

The computation is consistent if, 

1. $S_1$ is the start state.
2. For all $i$, state $S_{i+1}$ is among the possible allowed next states when in state $S_i$. I.e.:
$S_{i+1} \in\delta(S_i,w_i)$.



### Viewpoints on nondeterminism 

Three possible viewpoints:

1. Merlin model. Merlin gives Arthur the answer: an epsilon insertion of the string and a list of states. Arthur verifies. Merlin is always helpful and truthful. If there is an answer, Merlin will give it. You can either think of Arthur receving the full answers at first, or Merlin advising him at place there is a decision.
2. Hercules model. One by one Hercules tries all possible computations consistent with the all possible epsilon insertions until he he finds one that accepts, or has exhausted all possibilities. Hercules might arrange the computation as a tree of possibilities, and do either breath first or depth first search of the tree.
3. Schroedinger model: Multiple universes. We evolve a state set according to all possible worlds consistent with an epsilon insertion up to the n-th element of the string. This is the model we take, because it makes the NFA a DFA on the power set of states.




## NFA in Python

In [13]:
verbose = True
allow_broken = True

class MachineModelNFA:
    """
    A machine description is a dictionary with,
        states: a list of states
        alphabet: a list of letters, and implicitly the empty letter ':'
        transitions: a dictionary with keys tuples (a state,a letter) to a set of states
        start: a state (the start state)
        accept: a list of states (the accepting states)
        
        the current_state is a set of states
        
    """
    
    def __init__(self,machine_description):
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        # enclosing in a list absorbs the ennumerate
        self.current_state = self.epsilon_close(set([self.start_state])) 
        # the : is reserved
        assert ':' not in self.alphabet
    
    def epsilon_one_step(self,state_set):
        e = set()
        for state in state_set:
            t = (state,':')
            if t in self.transitions:
                e = e.union(self.transitions[t])
        return e.union(state_set)
    
    def epsilon_close(self,state_set):
        assert(allow_broken)
        # this is broken because an epsilon closer follows 0 or more epsilon 
        # moves; here we only follow 0 or 1 epsilon moves.
        return self.epsilon_one_step(state_set)


    def do_transition(self,letter):
        new_current = set()
        for state in self.current_state:
            if (state,letter) in self.transitions:
                new_current = new_current.union(self.transitions[(state,letter)])
        self.current_state = self.epsilon_close(new_current)
    
    def compute(self,word):
        self.current_state = self.epsilon_close(set([self.start_state]))
        if verbose : print("initial state:", self.current_state)
        for w in word:
            self.do_transition(w)
            if verbose : print("char:",w,"state:",self.current_state)
        return len(self.current_state.intersection(self.accept_states))>0


class TestMachineNFA:
    """
    A class that constructs a MachineModelNFA from a Machine Description, 
    and can run a test. The test object is a list of pairs, 
        [(string,case), ... ]
    where case is True or False, if the string is, or is not, in the 
    language.
    """
    
    def __init__(self,machine_description):
        self.machine = MachineModelNFA(machine_description)
      
    def run(self,tests):
        print('running tests ...')
        for (t,r) in (tests):
            if self.machine.compute(t) != r:
                print(r,'\t|'+t+'|','\tWRONG, ABORT')
                return False
            print(r,'\t|'+t+'|','\tOK')
        return True


## Exercise  A:

Use the MachineModelNFA to define NFA's to recognize the following languages over alphabet $\{\,0,1\,\}$ with the specified number of states. (Taken from Sipser 2nd edition exercise 1.7.)

1. The language $\{\,0\,\}$ with two states.
1. The language $0^*1^*0^+$ with three states.
1. The language $\{\,\epsilon\,\}$ with one state.
1. The language $0^*$ with one state.



In [40]:
# the following templates are provided.

nfa_ea = [None for i in range(5)]

nfa_ea[1] = {
    'states': ['O','Z'],
    'alphabet': ['0','1'],
    'transitions': {('Z','0'):['Z'],('O','1'):['O'],
                    ('O','0'):['O'],('Z','1'):['O'],
                   
                   },
    'start': 'Z',
    'accept': ['Z']
}

nfa_ea[2] = {
    'states': ['One','Two','Three'],
    'alphabet': ['0','1'],
    'transitions': {('One','0'):['One'],('One','1'):['Two'],
                    ('Two','0'):['Three'],('Two','1'):['Two'],
                     ('Three','0'):['Three']
                   },
    'start': 'One',
    'accept': ['Three','One']
}

nfa_ea[3] = {
    'states': ['E'],
    'alphabet': ['0','1'],
    'transitions': {('',''):['']},
    'start': 'E',
    'accept': ['E']
}

nfa_ea[4] = {
    'states': ['O'],
    'alphabet': ['0','1'],
    'transitions': {('O','0'):['O']},
    'start': 'O',
    'accept': ['O']
}


# testing
# these are basic tests only. 

nfa_test = [None for i in range(5)]

nfa_test[1] = [
    ('0000',True),
    ('1000',False)    
]
nfa_test[2] = [
    ('010',True),
    ('01011',False)    
]
nfa_test[3] = [
    ('',True),
    ('001',False)    
]
nfa_test[4] = [
    ('000',True),
    ('100',False)    
]



In [41]:

def basic_test(md,test):
    
    correct = 0
    num_tests = len(test)
    for i in range(1,num_tests):
        print("\nExercise",i)
        try:
            tm = TestMachineNFA(md[i])
            if tm.run(test[i]):
                correct += 1
        except Exception as exception:
            print("\n*** exception thrown:", str(type(exception)))

    print("correct:",correct,"out of",num_tests-1)

 
## testing exercise A 
basic_test(nfa_ea,nfa_test)
   


Exercise 1
running tests ...
initial state: {'Z'}
char: 0 state: {'Z'}
char: 0 state: {'Z'}
char: 0 state: {'Z'}
char: 0 state: {'Z'}
True 	|0000| 	OK
initial state: {'Z'}
char: 1 state: {'O'}
char: 0 state: {'O'}
char: 0 state: {'O'}
char: 0 state: {'O'}
False 	|1000| 	OK

Exercise 2
running tests ...
initial state: {'One'}
char: 0 state: {'One'}
char: 1 state: {'Two'}
char: 0 state: {'Three'}
True 	|010| 	OK
initial state: {'One'}
char: 0 state: {'One'}
char: 1 state: {'Two'}
char: 0 state: {'Three'}
char: 1 state: set()
char: 1 state: set()
False 	|01011| 	OK

Exercise 3
running tests ...
initial state: {'E'}
True 	|| 	OK
initial state: {'E'}
char: 0 state: set()
char: 0 state: set()
char: 1 state: set()
False 	|001| 	OK

Exercise 4
running tests ...
initial state: {'O'}
char: 0 state: {'O'}
char: 0 state: {'O'}
char: 0 state: {'O'}
True 	|000| 	OK
initial state: {'O'}
char: 1 state: set()
char: 0 state: set()
char: 0 state: set()
False 	|100| 	OK
correct: 4 out of 4


## Exercise B:

Use the general construction for constructing machines by concatentation, union and star to create NFA's for the following languages over $\Sigma = \{\,a,b\,\}$. (Taken from Sipser 2nd edition exercises 1.19 and 1.20.)

1. $a\,(b\,a)^*b$
1. $(\epsilon\cup a)\,b$
1. $\Sigma^*a\,\Sigma^*b\,\Sigma^*a\,\Sigma^*$
1. $(a\cup ba \cup bb)\Sigma^*$
1. $(a\cup b)^*a\,a\,a\,(a\cup b)^*$
1. $(((a\,a)^*(b\,b))\cup a\,b)^*$


In [99]:
# the following templates are provided.

nfa_eb = [None for i in range(7)]

nfa_eb[1] = {
    'states': ['E','Q','W','Wo','E','R'],
    'alphabet': ['a','b'],
    'transitions': {('E','a'):['Q'],('Q','b'):['R'],
                    ('R','a'):['W'],('W','b'):['R'],
                    
                   },
    'start': 'E',
    'accept': ['R']
}

nfa_eb[2] = {
    'states': ['E','A','B'],
    'alphabet': ['a','b'],
    'transitions': {('E','a'):['A'],('E','b'):['B'],
                    ('A','b'):['B'],
                   },
    'start': 'E',
    'accept': ['B']
}

nfa_eb[3] = {
    'states': ['E','M1','M2','M3','M4','M5','M6','M7','M8','M9'],
    'alphabet': ['a','b'],
    'transitions': {('E','a'):['M1'],('E','b'):['M2'],('E','a'):['M3'],
                    ('M1','a'):['M1'],('M2','b'):['M2'],
                     ('M1','a'):['M3'],('M2','a'):['M3'],
                    ('M3','a'):['M4'],('M3','b'):['M5'],('M3','b'):['M6'],
                    ('M4','a'):['M4'],('M5','b'):['M5'],
                    ('M4','b'):['M6'],('M5','b'):['M6'],
                    ('M6','a'):['M7'],('M6','b'):['M8'],('M6','a'):['M9'],
                    ('M7','a'):['M7'],('M8','b'):['M8'],
                    ('M7','a'):['M9'],('M8','a'):['M9'],
                    ('M9','a'):['M10'],('M9','b'):['M11'],
                    ('M10','a'):['M10'],('M11','b'):['M11'],
                   
                   },
    'start': 'E',
    'accept': ['M9','M10','M11']
}

nfa_eb[4] = {
    'states': ['E','M1','M2','ea','eb'],
    'alphabet': ['a','b','ba','bb'],
    'transitions': {('E','a'):['M2'],('E','b'):['M1'],
                    ('M1','a'):['M2'],('M1','b'):['M2'],
                    ('M2','a'):['ea'],('M2','b'):['eb'],
                    ('ea','a'):['ea'],('eb','b'):['eb'],
                   
                   
                   },
    'start': 'E',
    'accept': ['M2','ea','eb']
}

nfa_eb[5] = {
    'states': ['E','eb','ea','a1','a2','a3','eaa','ebb'],
    'alphabet': ['a','b'],
    'transitions': {('E','b'):['eb'],('eb','b'):['eb'],
                    ('E','a'):['ea'],('ea','a'):['ea'],
                    ('E','a'):['a1'],('a1','a'):['a2'],
                    ('a2','a'):['a3'],
                    ('ea','a'):['a1'],
                    ('eb','a'):['a1'],('a1','a'):['a2'],
                    ('a2','a'):['a3'],
                    ('a3','a'):['eaa'],('eaa','a'):['eaa'],
                    ('a3','b'):['ebb'],('ebb','b'):['ebb'],
        
        
        
        },
            


    'start': 'E',
    'accept': ['a3','eaa','ebb']
}

nfa_eb[6] = {
    'states': ['E','ea1','eaa','eb1','ebb','eab','ea2','b1','bb1'],
    'alphabet': ['a','b'],
    'transitions': {('E','a'):['ea1'],('E','b'):['eb1'],
                    ('E','a'):['ea2'],('ea2','a'):['eaa'],
                    ('ea1','a'):['eaa'],('ea2','b'):['eab'],
                    ('eaa','a'):['ea1'],('eaa','b'):['b1'],
                    ('b1','b'):['bb1'],('bb1','a'):['ea1'],
                    ('eab','a'):['ea2'],
                    ('eb1','b'):['ebb'],('ebb','b'):['eb1'],
                    
              
                   
                   
                   
                   
                   },
    'start': 'E',
    'accept': ['E','ebb','eab','bb1']
}



# testing
# these are basic tests only. 

nfa_test = [None for i in range(7)]

nfa_test[1] = [
    ('aba',False),
    ('ab',True),
    ('ababababab',True)
]
nfa_test[2] = [
    ('b',True),
    ('ab',True),
    ('bb',False),
    ('abb',False)
]
nfa_test[3] = [
    ('aba',True),
    ('a',False),
    ('aabb',False),
    ('bbabab',True)
]
nfa_test[4] = [
    ('a',True),
    ('ba',True),
    ('bb',True),
    ('b',False)
]
nfa_test[5] = [
    ('aaaaaaaaa',True),
    ('aaba',False)    
]
nfa_test[6] = [
    ('aabbaabb',True),
    ('bbbb',True),
    ('abaa',False)
]



In [100]:
# testing exercise B

basic_test(nfa_eb,nfa_test)


Exercise 1
running tests ...
initial state: {'E'}
char: a state: {'Q'}
char: b state: {'R'}
char: a state: {'W'}
False 	|aba| 	OK
initial state: {'E'}
char: a state: {'Q'}
char: b state: {'R'}
True 	|ab| 	OK
initial state: {'E'}
char: a state: {'Q'}
char: b state: {'R'}
char: a state: {'W'}
char: b state: {'R'}
char: a state: {'W'}
char: b state: {'R'}
char: a state: {'W'}
char: b state: {'R'}
char: a state: {'W'}
char: b state: {'R'}
True 	|ababababab| 	OK

Exercise 2
running tests ...
initial state: {'E'}
char: b state: {'B'}
True 	|b| 	OK
initial state: {'E'}
char: a state: {'A'}
char: b state: {'B'}
True 	|ab| 	OK
initial state: {'E'}
char: b state: {'B'}
char: b state: set()
False 	|bb| 	OK
initial state: {'E'}
char: a state: {'A'}
char: b state: {'B'}
char: b state: set()
False 	|abb| 	OK

Exercise 3
running tests ...
initial state: {'E'}
char: a state: {'M3'}
char: b state: {'M6'}
char: a state: {'M9'}
True 	|aba| 	OK
initial state: {'E'}
char: a state: {'M3'}
False 	|a| 	OK
in

## Exercise C:

There is a bug with the MachineModelNFA code, which is demonstarted with the following implementation of the language $((a\,b)^*\cup (b\,b)^*)^*$. Please fix the MachineModelNFA code. While this demonstrates the bug, your fix must completely fix the bug.

In [103]:
nfa_ec = {
    'states': ['S0','S1','E0','E1','E2','E3','F0','F1','F2','F3',],
    'alphabet': ['a','b'],
    'transitions': {
        ('S0','a'):['S1'],('S1','b'):['E0'],('S0','b'):['S2'],('S2','b'):['F0'],
        ('E0','a'):['E1'],('E1','b'):['E0'],
        ('F0','b'):['F1'],('F1','b'):['F0'],
    },
    'start': 'S0',
    'accept': ['S0','E0','F0',]
}


nfa_test = [None for i in range(5)]

nfa_test = [
    ('',True),
    ('ab',True),
    ('bb',True),
    ('aa',False),
    ('ba',False),
    ('abab',True),
    ('abbb',False),
    ('bbab',False),
    ('bbbb',True),
    ('abaa',False),
    ('bbba',False),
    ('aaaa',False),
    ('aaab',False),
]


tm = TestMachineNFA(nfa_ec)
tm.run(nfa_test)



running tests ...
initial state: {'S0'}
True 	|| 	OK
initial state: {'S0'}
char: a state: {'S1'}
char: b state: {'E0'}
True 	|ab| 	OK
initial state: {'S0'}
char: b state: {'S2'}
char: b state: {'F0'}
True 	|bb| 	OK
initial state: {'S0'}
char: a state: {'S1'}
char: a state: set()
False 	|aa| 	OK
initial state: {'S0'}
char: b state: {'S2'}
char: a state: set()
False 	|ba| 	OK
initial state: {'S0'}
char: a state: {'S1'}
char: b state: {'E0'}
char: a state: {'E1'}
char: b state: {'E0'}
True 	|abab| 	OK
initial state: {'S0'}
char: a state: {'S1'}
char: b state: {'E0'}
char: b state: set()
char: b state: set()
False 	|abbb| 	OK
initial state: {'S0'}
char: b state: {'S2'}
char: b state: {'F0'}
char: a state: set()
char: b state: set()
False 	|bbab| 	OK
initial state: {'S0'}
char: b state: {'S2'}
char: b state: {'F0'}
char: b state: {'F1'}
char: b state: {'F0'}
True 	|bbbb| 	OK
initial state: {'S0'}
char: a state: {'S1'}
char: b state: {'E0'}
char: a state: {'E1'}
char: a state: set()
False 	|

True

------

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

------